In [7]:
import time

count = 0
sleepy_time = 10
status_update_interval_in_min = 10

while (True):
    
    time.sleep(sleepy_time)
    count+=1
    
    if (sleepy_time * count) >= status_update_interval_in_min:
        print("Here!")

Here!
Here!


KeyboardInterrupt: 

In [5]:
txt_filepath = 'logs/status.txt'


def read_txt_file(txt_filepath):
    with open(txt_filepath, "r") as f:
        
        return f.read()
    

read_txt_file(txt_filepath)

'2025-03-13 12:13:38.806596 : DH Transcription Service started up! \n\n2025-03-13 12:13:38.806596 : DH Transcription Service started up! \n\n2025-03-13 12:13:38.806596 : DH Transcription Service started up! \n\n2025-03-13 12:13:38.806596 : DH Transcription Service started up! \n\n2025-03-13 12:13:38.806596 : DH Transcription Service started up! \n\n2025-03-13 12:13:38.806596 : DH Transcription Service started up! \n'

In [62]:
import os
import io
import re
import tempfile
import soundfile as sf

from google.auth.transport.requests import Request # type: ignore
from google.oauth2.credentials import Credentials # type: ignore
from google_auth_oauthlib.flow import InstalledAppFlow # type: ignore
from googleapiclient.discovery import build # type: ignore
from googleapiclient.http import MediaIoBaseDownload # type: ignore

from moviepy.video.io.VideoFileClip import VideoFileClip

# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']

# Hard-coded inputs
DRIVE_URL = 'https://drive.google.com/drive/u/1/folders/1kDhkf_o32tdPyCVrPshAE02dFZlSQCcd'
DOWNLOAD_FOLDER = 'downloads'
OUTPUTS_FOLDER = 'outputs'
SAMPLE_RATE = 16000

def authenticate():
    """Authenticate the user and return the service object."""
    creds = None
    credentials_path = 'credentials.json'
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    service = build('drive', 'v3', credentials=creds)
    return service

def extract_root_folder_id(drive_url):
    """Extract the folder ID from a Google Drive folder URL."""
    match = re.search(r'/folders/([a-zA-Z0-9_-]+)', drive_url)
    if not match:
        raise ValueError('Invalid Google Drive folder URL')
    return match.group(1)

def get_all_audio_files(folder_id, service):
    """Extract name and ids of all wav and mp3 (mpeg) files in folder id"""

    query = f"'{folder_id}' in parents and (mimeType='audio/wav' or mimeType='audio/mpeg')"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])

    list_of_wav = []
    for file in files:
        wav_entry = {'name':file['name'], 'id':file['id']}
        list_of_wav.append(wav_entry)
    
    return list_of_wav

def get_all_mp4_files(folder_id, service):
    """Extract name and ids of all mp4 files in folder id"""

    query = f"'{folder_id}' in parents and mimeType='video/mp4'"
    results = service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])

    list_of_mp4 = []
    for file in files:
        wav_entry = {'name':file['name'], 'id':file['id']}
        list_of_mp4.append(wav_entry)
    
    return list_of_mp4

def download_file(file_id, output_folder, service):
    ''' Download file from Google Drive'''
    
    file_metadata = service.files().get(fileId=file_id).execute()
    file_name = file_metadata['name']
    print(file_metadata)
    
    request = service.files().get_media(fileId = file_id)
    
    file_path = os.path.join(output_folder, file_name)
    os.makedirs(output_folder, exist_ok=True)
    
    with io.FileIO(file_path, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download {int(status.progress() * 100)}% complete.")
            print("done",done)
            
    return file_path
            
def check_if_file_in_folder(filename, folder_path):
    ''' check if a file is in a folder , return True if in, return False if not '''
    
    onlyfiles = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    
    if (filename in onlyfiles):
        print(f"{filename} is in {folder_path}")
        return True
    else:
        print(f"{filename} is not in {folder_path}")
        return False
    
def export_text_to_txt(text, output_text_filepath):
    
    with open(output_text_filepath, "w") as f:
        
        f.write(text)
        
def audio_from_mp4(mp4_filepath):
    ''' Read an mp4's audio file into a numpy array'''
    
    video_clip = VideoFileClip(mp4_filepath)

    if not video_clip.audio:
        return {"error": "No audio found in the MP4 file"}

    audio_clip = video_clip.audio
    audio_array = audio_clip.to_soundarray()
    sample_rate = audio_clip.fps

    audio_clip.close()
    video_clip.close()

    os.remove(mp4_filepath)

    audio_array = audio_array.T

    return audio_array, sample_rate

from asr_inference_service.model import ASRModelForInference
from asr_inference_service.audio_preprocessing import resample_audio_array

model = ASRModelForInference(
    model_dir="pretrained_models/whisper-large-v3",
    sample_rate=16000,
    device="cuda",
    timestamp_format="seconds",
    min_segment_length=0.5,
    min_silence_length=9999999999,
)

service = authenticate()
root_folder_id = extract_root_folder_id(DRIVE_URL)
list_of_audio_in_drive = get_all_audio_files(root_folder_id, service)
list_of_mp4_in_drive = get_all_mp4_files(root_folder_id, service)

for audio in list_of_audio_in_drive:
    if not check_if_file_in_folder(audio['name'], DOWNLOAD_FOLDER):
        output_filepath = download_file(audio['id'], 'downloads', service)
        transcriptions = model.diar_inference(filepath=output_filepath)
        pre, ext = os.path.splitext(audio['name'])
        export_text_to_txt(transcriptions, os.path.join(OUTPUTS_FOLDER, pre+'.txt'))
        

def audio_from_mp4(mp4_filepath):
    ''' Read an mp4's audio file into a numpy array'''
    
    video_clip = VideoFileClip(mp4_filepath)

    if not video_clip.audio:
        return {"error": "No audio found in the MP4 file"}

    audio_clip = video_clip.audio
    audio_array = audio_clip.to_soundarray()
    sample_rate = audio_clip.fps

    audio_clip.close()
    video_clip.close()

    audio_array = audio_array.T

    return audio_array, sample_rate

for video in list_of_mp4_in_drive:
    if not check_if_file_in_folder(video['name'], DOWNLOAD_FOLDER):
        output_filepath = download_file(video['id'], 'downloads', service)
        numpy_audio_array, sample_rate = audio_from_mp4(output_filepath)
        numpy_audio_array = resample_audio_array(numpy_audio_array, sample_rate, SAMPLE_RATE)
        
        with tempfile.NamedTemporaryFile(delete=True, suffix=".wav") as temp_file:
            sf.write(temp_file, numpy_audio_array, SAMPLE_RATE)
            temp_file_path = temp_file.name
            transcription = model.diar_inference(temp_file_path)
        
        print(transcription)
        pre, ext = os.path.splitext(video['name'])
        export_text_to_txt(transcription, os.path.join(OUTPUTS_FOLDER, pre+'.txt'))

INFO | 2025-03-13 09:27:26,963 | Loading Whisper ASR model...
INFO | 2025-03-13 09:27:26,964 | Torch dtype: torch.float16
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO | 2025-03-13 09:27:29,519 | Models loaded. Elapsed time: 2.5548432620125823
INFO | 2025-03-13 09:27:29,520 | Running on device: cuda
INFO | 2025-03-13 09:27:29,520 | Running on device: cuda
INFO | 2025-03-13 09:27:29,520 | Minimum Segment Length: 0.5
INFO | 2025-03-13 09:27:29,520 | Minimum Silence Length: 9999999999
INFO | 2025-03-13 09:27:31,185 | Pyannote model loaded!


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=611128439745-oqouj3m2h94stpi9a9chnkq1o7b5lsea.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A53471%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=x8p34msKAqfBfOVk92BvTUnJZ1SUHc&access_type=offline


INFO | 2025-03-13 09:27:37,114 | "GET /?state=x8p34msKAqfBfOVk92BvTUnJZ1SUHc&code=4/0AQSTgQHgEwxF92GSLEGug42fQNgPMZZCfccz4JD7iSxM1LFKBKRJpBchu0NxbB4kuaiElw&scope=https://www.googleapis.com/auth/drive HTTP/1.1" 200 65
INFO | 2025-03-13 09:27:37,231 | file_cache is only supported with oauth2client<4.0.0


Rick Astley - Never Gonna Give You Up (Official Music Video).mp3 is in downloads
steroids_120sec.wav is in downloads
sample_1.mp4 is not in downloads
{'kind': 'drive#file', 'id': '1mlfWQ6hB4QCE6itmSDPchtDPi06GF1Dp', 'name': 'sample_1.mp4', 'mimeType': 'video/mp4'}
Download 100% complete.
done True
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso6iso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 965, 'fps': 60.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file

INFO | 2025-03-13 09:27:44,743 | Audio preprocessing starte : 44100 SR to 16000 SR
INFO | 2025-03-13 09:27:44,825 | Resample done
INFO | 2025-03-13 09:27:44,826 | Audio preprocessed, Shape : (1297920,)
INFO | 2025-03-13 09:27:44,836 | Diarization Model triggered.
INFO | 2025-03-13 09:27:44,837 | Diarization started
/home/digitalhub/Desktop/ian_projects/PASSIONS/google_drive_listener/venv/lib/python3.11/site-packages/pyannote/audio/models/blocks/pooling.py:104: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  std = sequences.std(dim=-1, correction=1)
INFO | 2025-03-13 09:27:46,757 | Diarization Model Done. Elapsed time: 1.9211871160077862
/home/digitalhub/Desktop/ian_projects/PASSIONS/google_drive_listener/venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The 

[2.81 - 3.78] [SPEAKER_00] :  What law do you like?

[5.09 - 8.35] [SPEAKER_01] :  I like the traffic law of wearing the helmet when traveling by motorbike.

[10.80 - 12.27] [SPEAKER_00] :  Is that an international law?

[13.97 - 15.81] [SPEAKER_01] :  People around the world follow this law.

[16.99 - 18.04] [SPEAKER_00] :  When was it issued?

[19.17 - 21.73] [SPEAKER_01] :  I'm not quite sure, but I guess a long time ago.

[22.81 - 23.94] [SPEAKER_00] :  Who told you that law?

[25.04 - 26.05] [SPEAKER_01] :  I learned it at school.

[27.22 - 28.77] [SPEAKER_00] :  Is it easy to follow that law?

[29.88 - 34.03] [SPEAKER_01] :  Yes, it is. It doesn't cause any inconvenience at all.

[35.06 - 36.52] [SPEAKER_00] :  What do you think about that law?

[37.58 - 41.48] [SPEAKER_01] :  The traffic law is so necessary for traffic participants when they are on the road.

[43.97 - 46.30] [SPEAKER_00] :  What benefit can you get from following that law?

[47.33 - 49.54] [SPEAKER_01] :  It may

In [63]:
import os
import io
import re
import tempfile
import soundfile as sf

from google.auth.transport.requests import Request # type: ignore
from google.oauth2.credentials import Credentials # type: ignore
from google_auth_oauthlib.flow import InstalledAppFlow # type: ignore
from googleapiclient.discovery import build # type: ignore
from googleapiclient.http import MediaIoBaseDownload # type: ignore
from asr_inference_service.audio_preprocessing import resample_audio_array

from moviepy.video.io.VideoFileClip import VideoFileClip

SAMPLE_RATE = 16000

def audio_from_mp4(mp4_filepath):
    ''' Read an mp4's audio file into a numpy array'''
    
    video_clip = VideoFileClip(mp4_filepath)

    if not video_clip.audio:
        return {"error": "No audio found in the MP4 file"}

    audio_clip = video_clip.audio
    audio_array = audio_clip.to_soundarray()
    sample_rate = audio_clip.fps

    audio_clip.close()
    video_clip.close()

    audio_array = audio_array.T

    return audio_array, sample_rate

from googleapiclient.http import MediaFileUpload

def upload_txt_file(file_path, folder_id, service):
    """Uploads a TXT file to a specific Google Drive folder."""
    
    file_metadata = {
        "name": os.path.basename(file_path),  # Keep the original file name
        "parents": [folder_id]  # Upload to the specified folder
    }

    media = MediaFileUpload(file_path, mimetype="text/plain")

    uploaded_file = service.files().create(
        body=file_metadata,
        media_body=media,
        fields="id, name"
    ).execute()

    print(f"Uploaded {uploaded_file['name']} with ID: {uploaded_file['id']}")
    return 

for video in list_of_mp4_in_drive:
    if not check_if_file_in_folder(video['name'], DOWNLOAD_FOLDER):
        output_filepath = download_file(video['id'], 'downloads', service)
        numpy_audio_array, sample_rate = audio_from_mp4(output_filepath)
        numpy_audio_array = resample_audio_array(numpy_audio_array, sample_rate, SAMPLE_RATE)
        
        with tempfile.NamedTemporaryFile(delete=True, suffix=".wav") as temp_file:
            sf.write(temp_file, numpy_audio_array, SAMPLE_RATE)
            temp_file_path = temp_file.name
            transcription = model.diar_inference(temp_file_path)
        
        print(transcription)
        pre, ext = os.path.splitext(video['name'])
        output_txt_path = os.path.join(OUTPUTS_FOLDER, pre+'.txt')
        export_text_to_txt(transcription, output_txt_path)
        upload_txt_file(output_txt_path, root_folder_id, service)
        

#TODO ### GET PERMISSIONS TO WRITE #### TODO

sample_1.mp4 is not in downloads
{'kind': 'drive#file', 'id': '1mlfWQ6hB4QCE6itmSDPchtDPi06GF1Dp', 'name': 'sample_1.mp4', 'mimeType': 'video/mp4'}
Download 100% complete.
done True
{'video_found': True, 'audio_found': True, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso6iso2avc1mp41', 'encoder': 'Lavf60.16.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [1280, 720], 'bitrate': 965, 'fps': 60.0, 'codec_name': 'h264', 'profile': '(High)', 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}, {'input_number': 0, 'stream_number': 1, 'stream_type': 'audio', 'language': 'eng', 'default': True, 'fps': 44100, 'bitrate': 127, 'metadata': {'Metadata': '', 'handler_name': 'ISO Media file produced by Google Inc.', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 81.12, 'bitrate': 1098, '

INFO | 2025-03-13 09:28:22,796 | Audio preprocessing starte : 44100 SR to 16000 SR
INFO | 2025-03-13 09:28:22,877 | Resample done
INFO | 2025-03-13 09:28:22,878 | Audio preprocessed, Shape : (1297920,)
INFO | 2025-03-13 09:28:22,886 | Diarization Model triggered.
INFO | 2025-03-13 09:28:22,886 | Diarization started
INFO | 2025-03-13 09:28:24,757 | Diarization Model Done. Elapsed time: 1.871639209013665
/home/digitalhub/Desktop/ian_projects/PASSIONS/google_drive_listener/venv/lib/python3.11/site-packages/transformers/models/whisper/generation_whisper.py:480: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
INFO | 2025-03-13 09:28:25,359 | Inference Model triggered. Elapsed time: 0.6003430539858527
INFO | 2025-03-13 09:28:26,154 | Inference Model triggered. Elapsed time: 0.795258702011779
INFO | 2025-03-13 09:28:26,759 | Inference Model triggered. Elapsed time: 0.6039215910132043
INFO | 2025-03-13 09:28:27,403 | Infe

[2.81 - 3.78] [SPEAKER_00] :  What law do you like?

[5.09 - 8.35] [SPEAKER_01] :  I like the traffic law of wearing the helmet when traveling by motorbike.

[10.80 - 12.27] [SPEAKER_00] :  Is that an international law?

[13.97 - 15.81] [SPEAKER_01] :  People around the world follow this law.

[16.99 - 18.04] [SPEAKER_00] :  When was it issued?

[19.17 - 21.73] [SPEAKER_01] :  I'm not quite sure, but I guess a long time ago.

[22.81 - 23.94] [SPEAKER_00] :  Who told you that law?

[25.04 - 26.05] [SPEAKER_01] :  I learned it at school.

[27.22 - 28.77] [SPEAKER_00] :  Is it easy to follow that law?

[29.88 - 34.03] [SPEAKER_01] :  Yes, it is. It doesn't cause any inconvenience at all.

[35.06 - 36.52] [SPEAKER_00] :  What do you think about that law?

[37.58 - 41.48] [SPEAKER_01] :  The traffic law is so necessary for traffic participants when they are on the road.

[43.97 - 46.30] [SPEAKER_00] :  What benefit can you get from following that law?

[47.33 - 49.54] [SPEAKER_01] :  It may

In [23]:
print(list_of_mp4_in_drive)
print(list_of_audio_in_drive)

[{'name': 'sample_1.mp4', 'id': '1mlfWQ6hB4QCE6itmSDPchtDPi06GF1Dp'}]
[{'name': 'Rick Astley - Never Gonna Give You Up (Official Music Video).mp3', 'id': '1XHIMy9qpFT5pF77gIUwSjvERdhfUP9K6'}, {'name': 'steroids_120sec.wav', 'id': '1EmDXVpJ9_VUAs3MSpOPAd8xlYjwBUlIF'}]


In [ ]:
from asr_inference_service.model import ASRModelForInference

model = ASRModelForInference(
    model_dir=os.environ["PRETRAINED_MODEL_DIR"],
    sample_rate=int(os.environ["SAMPLE_RATE"]),
    device=os.environ["DEVICE"],
    timestamp_format=os.environ["TIMESTAMPS_FORMAT"],
    min_segment_length=float(os.environ["MIN_SEGMENT_LENGTH"]),
    min_silence_length=float(os.environ["MIN_SILENCE_LENGTH"]),
)

model.diar_inference()

SyntaxError: invalid syntax (2425223809.py, line 1)

In [9]:

from moviepy.video.io.VideoFileClip import VideoFileClip

path_to_mp4 = "/home/digitalhub/Desktop/ian_projects/HR/audio/GMT20250113-075636_Recording_1280x720.mp4"
def audio_from_mp4(mp4_filepath):
    ''' Read an mp4's audio file into a numpy array'''
    
    video_clip = VideoFileClip(mp4_filepath)

    if not video_clip.audio:
        return {"error": "No audio found in the MP4 file"}

    audio_clip = video_clip.audio
    audio_array = audio_clip.to_soundarray()
    sample_rate = audio_clip.fps

    audio_clip.close()
    video_clip.close()

    audio_array = audio_array.T

    return audio_array, sample_rate

audio_from_mp4(path_to_mp4)

OSError: Error passing `ffmpeg -i` command output:

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from '/home/digitalhub/Desktop/ian_projects/HR/audio/GMT20250113-075636_Recording_1280x720.mp4':
  Metadata:
    major_brand     : mp42
    minor_version   : 0
    compatible_brands: isommp42
    creation_time   : 2025-01-13T07:56:36.000000Z
  Duration: 02:06:11.96, start: 0.000000, bitrate: 755 kb/s
  Chapters:
    Chapter #0:0: start 0.000000, end 7571.960000
      Metadata:
        title           : Recording Started
  Stream #0:0[0x1](und): Audio: aac (LC) (mp4a / 0x6134706D), 32000 Hz, mono, fltp, 126 kb/s (default)
      Metadata:
        creation_time   : 2025-01-13T07:56:36.000000Z
        handler_name    : AAC audio
        vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(tv, bt470bg/bt470bg/bt709, progressive), 1280x720, 626 kb/s, 25 fps, 25 tbr, 30k tbn (default)
      Metadata:
        creation_time   : 2025-01-13T07:56:36.000000Z
   

In [14]:
from datetime import datetime

datetime.fromtimestamp(1345).strftime("%I:%M:%S")

'07:52:25'

In [25]:
start_time = 2.3
end_time = 5

start_hours = start_time // 3600
start_minutes = (start_time % 3600) // 60
start_seconds = start_time % 60

end_hours = end_time // 3600
end_minutes = (end_time % 3600) // 60
end_seconds = end_time % 60

segment_string = f"[{start_hours:02}:{start_minutes:02}:{start_seconds:02} - {end_hours:02}:{end_minutes:02}:{end_seconds:02}]\n\n"

In [26]:
segment_string

'[0.0:0.0:2.3 - 00:00:05]\n\n'